# 4. RAG Completo con Búsqueda Vectorial y LangChain

## Objetivos de Aprendizaje
- Integrar el chunking, los embeddings y la recuperación en un único pipeline de RAG.
- Utilizar una base de datos vectorial (FAISS) para almacenar y buscar chunks de manera eficiente.
- Construir una cadena `RetrievalQA` en LangChain para orquestar todo el proceso.
- Realizar una consulta y obtener una respuesta generada por el LLM basada en el contexto recuperado.

## El Pipeline de RAG Vectorial

En los notebooks anteriores, preparamos los componentes: dividimos el texto y creamos embeddings. Ahora, vamos a unirlos en un sistema funcional. Este es el flujo de trabajo completo de un RAG basado en vectores:

1.  **Indexación (se hace una sola vez):**
    - Se carga un documento.
    - Se divide en chunks.
    - Se generan embeddings para cada chunk.
    - Los chunks y sus embeddings se almacenan en una **base de datos vectorial** (Vector Store).
2.  **Recuperación y Generación (se hace para cada consulta):**
    - El usuario hace una pregunta (consulta).
    - Se genera un embedding para la consulta.
    - Se usa el embedding de la consulta para buscar en la base de datos vectorial los chunks más similares (búsqueda de similitud).
    - Los chunks recuperados se pasan como contexto al LLM junto con la consulta original.
    - El LLM genera una respuesta basada en el contexto proporcionado.

# Install faiss
!pip install faiss-cpu

In [3]:
import os
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from IPython.display import display, Markdown

# --- Configuración del Cliente y Modelos de LangChain ---
os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_API_BASE"] = os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com")

# Modelo de embeddings (compatible con la API de OpenAI)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# Modelo de lenguaje para la generación de respuestas
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.1
)

print("✓ Modelos de embeddings y chat inicializados con LangChain.")

✓ Modelos de embeddings y chat inicializados con LangChain.


## 1. Carga y Fragmentación del Documento

Reutilizamos el mismo texto sobre la historia de la IA y lo dividimos en chunks, exactamente como en el notebook anterior.

In [5]:
long_text = (
    "La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. "
    "Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. "
    "El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia de Dartmouth, considerada el nacimiento oficial de la IA como campo de estudio. "
    "Los primeros años estuvieron marcados por un gran optimismo, con la creación de programas como el Logic Theorist y el General Problem Solver, que podían resolver problemas de lógica y teoremas matemáticos. "
    "Sin embargo, las limitaciones computacionales y la complejidad de los problemas del mundo real llevaron al primer 'invierno de la IA' en la década de 1970, un período de reducción de fondos y escepticismo. "
    "El resurgimiento llegó en la década de 1980 con el auge de los sistemas expertos, programas que encapsulaban el conocimiento de un experto humano en un dominio específico, como el diagnóstico médico (por ejemplo, MYCIN). "
    "Estos sistemas demostraron el valor comercial de la IA, pero su fragilidad y el alto costo de mantenimiento condujeron a un segundo invierno a finales de los 80 y principios de los 90. "
    "La revolución moderna de la IA comenzó a gestarse a finales de los 90 y principios de los 2000, impulsada por tres factores clave: la disponibilidad de grandes volúmenes de datos (Big Data), el desarrollo de hardware más potente (especialmente las GPU) y los avances en algoritmos de aprendizaje automático, en particular las redes neuronales profundas (deep learning). "
    "Hitos como la victoria de Deep Blue de IBM sobre el campeón de ajedrez Garry Kasparov en 1997 y, más tarde, el triunfo de AlphaGo de DeepMind en el juego de Go en 2016, demostraron el poder del aprendizaje por refuerzo y el deep learning. "
    "Hoy, vivimos en la era de los modelos de lenguaje grande (LLM) como GPT y Claude, y los modelos de difusión para la generación de imágenes, que han llevado la IA a la corriente principal, transformando industrias y planteando nuevas preguntas sobre el futuro de la tecnología y la humanidad."
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=50,
    length_function=len
)
chunks = text_splitter.split_text(long_text)

display(Markdown(f"### 📜 Documento dividido en {len(chunks)} chunks."))

### 📜 Documento dividido en 7 chunks.

## 2. Creación de la Base de Datos Vectorial con FAISS

Aquí es donde la magia ocurre. LangChain simplifica enormemente la creación de la base de datos vectorial.

Usamos `FAISS.from_texts()`, que realiza los siguientes pasos internamente:
1.  Toma nuestra lista de `chunks`.
2.  Utiliza el modelo de `embeddings` que le proporcionamos para convertir cada chunk en un vector.
3.  Crea un índice FAISS en memoria con todos los vectores y sus correspondientes chunks de texto.

**FAISS (Facebook AI Similarity Search)** es una librería altamente optimizada para la búsqueda de similitud en conjuntos masivos de vectores.

In [6]:
try:
    # Crear la base de datos vectorial a partir de los chunks y el modelo de embeddings
    vector_db = FAISS.from_texts(texts=chunks, embedding=embeddings)
    print("✓ Base de datos vectorial FAISS creada en memoria.")
except Exception as e:
    print(f"❌ Error al crear la base de datos vectorial: {e}")

✓ Base de datos vectorial FAISS creada en memoria.


## 3. Consulta y Recuperación de Chunks Relevantes

Antes de pasar al LLM, veamos qué recupera nuestro sistema. Un `retriever` es un componente de LangChain que, dada una consulta, devuelve los documentos más relevantes desde la base de datos vectorial.

In [8]:
query = "¿Quién acuñó el término 'inteligencia artificial'?"

if 'vector_db' in locals():
    # El retriever es la interfaz para buscar en la base de datos
    retriever = vector_db.as_retriever(search_kwargs={"k": 2}) # k=2 para obtener los 2 chunks más relevantes
    
    # Realizar la búsqueda
    relevant_chunks = retriever.invoke(query)
    
    display(Markdown(f"### 🔍 Chunks recuperados para la consulta: *'{query}'*"))
    for i, chunk in enumerate(relevant_chunks):
        print(f"--- CHUNK RELEVANTE {i+1} ---")
        print(chunk.page_content)
        print()


### 🔍 Chunks recuperados para la consulta: *'¿Quién acuñó el término 'inteligencia artificial'?'*

--- CHUNK RELEVANTE 1 ---
La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia

--- CHUNK RELEVANTE 2 ---
John McCarthy en 1956 en la famosa Conferencia de Dartmouth, considerada el nacimiento oficial de la IA como campo de estudio. Los primeros años estuvieron marcados por un gran optimismo, con la creación de programas como el Logic Theorist y el General Problem Solver, que podían resolver problemas de lógica y teoremas matemáticos. Sin embargo, las



## 4. Generación de la Respuesta con `RetrievalQA`

Ahora, unimos todo. La cadena `RetrievalQA` de LangChain está diseñada exactamente para este propósito. Le proporcionamos:

-   `llm`: El modelo de lenguaje que generará la respuesta final.
-   `retriever`: Nuestro recuperador de la base de datos FAISS.
-   `chain_type="stuff"`: Esta es la estrategia más simple. Simplemente "rellena" (stuff) el prompt con todos los chunks recuperados.

La cadena se encargará de todo el proceso: tomar la consulta, recuperar los chunks, construir el prompt y obtener la respuesta del LLM.

In [9]:
if 'vector_db' in locals():
    # Crear la cadena de RetrievalQA
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever()
    )
    
    # Ejecutar la cadena con nuestra consulta
    response = qa_chain.invoke({"query": query})
    
    display(Markdown(f"### 💬 Respuesta Generada por el LLM"))
    display(Markdown(response['result']))

### 💬 Respuesta Generada por el LLM

El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 durante la famosa Conferencia de Dartmouth.

## 5. Conclusiones

¡Felicidades! Has construido un sistema de RAG completo y funcional. 

- **Automatización**: LangChain ha orquestado todos los pasos, desde la creación de la base de datos vectorial hasta la generación de la respuesta final, con muy poco código.
- **Precisión**: La respuesta del LLM se basa directamente en la información encontrada en el documento, lo que la hace precisa y fiable, evitando alucinaciones.
- **Eficiencia**: FAISS permite que la búsqueda de similitud sea extremadamente rápida, incluso con millones de documentos.

Este es el patrón fundamental sobre el que se construyen la mayoría de las aplicaciones de RAG modernas. A partir de aquí, se pueden explorar técnicas más avanzadas como diferentes estrategias de chunking, re-ranking de resultados o cadenas más complejas.

In [10]:
# Probemos con otra pregunta para ver la robustez del sistema
if 'qa_chain' in locals():
    query_invierno = "¿Qué causó el primer invierno de la IA?"
    response_invierno = qa_chain.invoke({"query": query_invierno})
    
    display(Markdown(f"### ❓ Consulta: *{query_invierno}*"))
    display(Markdown(f"### 💬 Respuesta: *{response_invierno['result']}*"))

### ❓ Consulta: *¿Qué causó el primer invierno de la IA?*

### 💬 Respuesta: *El primer "invierno de la IA" fue causado principalmente por las limitaciones computacionales y la complejidad de los problemas del mundo real. Aunque los primeros programas de IA, como el Logic Theorist y el General Problem Solver, lograron resolver problemas de lógica y teoremas matemáticos, pronto se hizo evidente que estos enfoques no podían escalar ni manejar situaciones más complejas fuera de entornos controlados. Esto llevó a una reducción de fondos y un aumento del escepticismo en la década de 1970, marcando así el primer invierno de la IA.*